# Preprocessing 

In questo script di preprocessing vengono estratti i volti dal dataset selezionato, successivamente viene effettuata la predizione dei 68 landamark facciali e su di essi viene applicato lo Spider Web Model che restituirà per ogni immagine rilevata un array caratteristico. Al termine dell'iterazione delle immagini all'interno di un file .csv verranno salvati l'array calcolato dal modello a ragnatela e le informazioni riguardo il genere e il rilevamento del volto.

## Import di librerie

In [1]:
!wget -O shape_predictor_68_face_landmarks.dat "https://github.com/italojs/facial-landmarks-recognition-/raw/master/shape_predictor_68_face_landmarks.dat"
!wget -O mmod_human_face_detector.dat "http://arunponnusamy.com/files/mmod_human_face_detector.dat"

import os
import time
import threading
import multiprocessing
import math
from pylab import *
import PIL.Image as im
import csv
import sys

from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
from string import Template
import string

from tqdm import tqdm
import pandas as pd

import dlib.cuda as cuda

dlib.DLIB_USE_CUDA = True

mmod = True #Usa rete neurale


## Funzione distanza euclidea

Questa funzione presi in input le coordinate di due punti P1(x1,y1) e P2(x2,y2) restituisce la loro distanza euclidea.

In [2]:
def distanza(x1, y1, x2, y2):    
    x12 = (x2 - x1) * (x2 - x1)
    y12 = (y2 - y1) * (y2 - y1)
    xy = x12 + y12
    dist = math.sqrt(xy)
    return dist

## Funzione aggiunta landmark su ragnatela

In questa funzione presi in input

- le coordinate del landmark che è sulla punta del naso
- il raggio (inizialmente uguale a 0)
- le coordinate del punto in questione
- la distanza euclidea tra il landmark sulla punta del naso e il punto in questione
- il coefficiente angolare

assegna al punto in questione il cerchio, il quadrante e la fetta e aggiunge il punto nella posizione corrispondente nell'array caratteristico.

In [3]:
def aggiungi(xcentro, ycentro, rax, xpunto, ypunto, distNaso, coeff):

    indice = 0

    settore = np.zeros(3) #cerchio, quadrante, fetta

    
    a = 0  # a = raggioStart
    b8 = 4 * rax / 10  # b = raggioStop
    i = 1  # in quale cerchio cade il punto. i = [1, cerchi]primo

    b4 = 7 * rax / 10
    b2 = 9 * rax / 10

    #cerchi
    if( distNaso > a and distNaso <= b8):
        # il punto appartiene al primo cerchio (quello più esterno)
        settore[0] = 1
    elif(distNaso > b8 and distNaso <= b4):
        # il punto appartiene al secondo cerchio
        settore[0] = 2
    elif(distNaso > b4 and distNaso <= b2):
        # il punto appartiene al terzo cerchio
        settore[0] = 3   
    else :
        # il punto appartiene al quarto cerchio
        settore[0] = 4
        


    #quadrante
    if (xpunto <= xcentro and y <= ycentro):
        # il punto appartiene al quadrante in alto a sinistra
        settore[1] = 2
    elif (x <= xnose and y >= ynose):
        # il punto appartiene al quadrante in basso a sinistra
        settore[1] = 3
    elif (x >= xnose and y <= ynose):
        # il punto appartiene al quadrante in alto a destra
        settore[1] = 1
    else:
        # il punto appartiene al quadrante in basso a destra
        settore[1] = 4

    
    a = 0                 #grado Start
    b = 90  / fetteQ      #grado Stop
    i = 1                 #in quale fetta cade il punto. i = [1, fette]

    radang_a = 0                    # radiante Start
    radang_b = math.radians(b)      # radiante Stop
    tng_a = math.tan(radang_a)
    tng_b = math.tan(radang_b)


    #fetta
    while(settore[2] == 0 and b < 90):
        if(coeff > tng_a and coeff <= tng_b):
            settore[2] = i
        b = b + (90  / fetteQ)
        radang_b = math.radians(b)  # radiante Stop
        tng_a = tng_b
        tng_b = math.tan(radang_b)
        i = i+1

    if(xpunto == xnose):
        settore[2] = 1


    if(settore[2] == 0):
        settore[2] = fetteQ

        # settore[0] = cerchio
        # settore[1] = quadrante
        # settore[2] = fetta



    if(settore[1] == 1 or settore[1] == 3):
        indice = int(fette * (settore[0]-1) + fetteQ * (settore[1] -1) + abs(settore[2] - 4 ) -1)
    else:
        indice = int(fette * (settore[0] - 1) + fetteQ * (settore[1] - 1) + settore[2] - 1)


    try:
        if (xnose != xpunto or ynose != ypunto):   #il naso non ha settore
            volto[indice] = int(volto[indice] + 1)
    except:
        #else:
        print("ERORE------")
        print("indice ", indice)

    return indice


## Inizializzazioni e lettura dei dataset

In questa sezione vengono inizializzate le variabili utili al rilevamento del volto, alla predizione dei landmark e allo Spider Web Model. Successivamente vengono letti come data frame Pandas i dataset e come file csv i file contenenti gli attributi d'interesse dei dataset.

In [4]:
#rilevatore MMOD basato su CNN
detector =  dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")


#crea il predittore dei landmarks facciali
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

intero = 0

voltostr = ''

cerchi = 4  # cerchi della ragnatela
fetteQ = 4  # fette per quadrante

fette = fetteQ * 4
s1 = cerchi * fette

#array di numeri 0 di dimensione s1=64 
volto = np.zeros(s1)

In [5]:
base_path='train'

folder_img = os.path.join(base_path, "data")
csv_file = os.path.join(base_path, 'train_gtruth.csv') #settaggio per Whe Dataset

dataset = pd.read_csv(csv_file)

vengono presi dai file csv solo i dati d'interesse ovvero l'id dell'immagine e il genere 

In [6]:
images=dataset[['TEMPLATE_ID']]
gender=dataset[['GENDER']]

num_volto = 0

n_imgs = images.shape[0]    #numero totale di immagini nel csv

inizializzazione dei dizionari per il vettore caratteristico dello spider web-model, per il genere e per il rilevamento del volto

In [7]:
dizionario = [ [ 0 for y in range(s1)] for x in range(n_imgs)]   #inizializzazione di n_imgs array ognuno di 64 elementi

dizionario_str = ['' for xx in range(n_imgs)]    #inizializzazione array dizionario_str

gender_str = ['' for xx in range(n_imgs)]    #inizializzazione array gender_str

valid_face = [0 for xx in range(n_imgs)]    #inizializzazione array valid_face

## Inizio ciclo di iterazione

In questa sezione inizia il ciclo di iterazione su tutte le immagini presenti nel dataset con funzione tqdm() per visualizzare una progress bar

In [8]:
for count,img in tqdm(images.iterrows(), total=n_imgs):
    
    #Settaggio per WheDataset
    if(gender.loc[count,'GENDER'] == 0):
        #1 = MASCHIO
        gender_str[num_volto]=1
    if(gender.loc[count,'GENDER'] == 1):
        #0 = DONNA
        gender_str[num_volto]=0


    filename = str(img[0])+'.jpg'

    filename = os.path.join(folder_img, filename)

    #lettura immagine con opencv
    foto = cv2.imread(filename)
    
    volto = np.zeros(s1)
    
    xnose = 0   #coordinata X del punto corrispondente al naso
    ynose = 0   #coordinata Y del punto corrispondente al naso
    

    xlont = 0 #coordinata x del landmark più lontano dal naso
    ylont = 0 #coordinata y del landmark più lontano dal naso

    #resize a 512px della foto
    foto = imutils.resize(foto, width=512) 

    #coversione immagine in scala di grigi       
    gray = cv2.cvtColor(foto, cv2.COLOR_BGR2GRAY)

    #rileva i volti nell'immagine in scala di grigi
    rects = detector(foto, 1)


    dista = 0
    raggio = 0

    m = 0 #coefficiente angolare
    d = 0 #distanza euclidea



    #per ogni volto (rect) trovato nella foto
    for (i, rect) in enumerate(rects):
        
        valid_face[num_volto]=1 #settato a 1 perchè ha trovato un volto
        
        
        if mmod:
            rect = rect.rect    #restituisce oggetto rect
        
        #determina i landmarks facciali per la regiione del volto
        shape = predictor(gray, rect)

        #converte i landmark facciali (x, y)-coordinates in un array di NumPy
        shape = face_utils.shape_to_np(shape)

        
        #converte i rettangoli di dlib in rettangoli di OpenCV 
        (x, y, w, h) = face_utils.rect_to_bb(rect)

        xnose = shape[33][0]    #assegna la coordinata X del landmark 33 che corrisponde alla punta del naso
        ynose = shape[33][1]    #assegna la coordinata Y del landmark 33 che corrisponde alla punta del naso

        
        #per ogni coppia di coordinate (x,y) all'interno dell'array di landmarks
        for (x, y) in shape:

            #calcola la distanza euclidea tra il punto del naso e i punti facciali dei landmark
            dista = distanza(xnose, ynose, x, y)
        
            
            if(dista > raggio):
                #nell'ultima iterazione il raggio sarà pari alla distanza euclidea tra il punto del naso (Xnose,Ynose)e il punto(landmark) più lontano
                raggio = dista
                xlont = x   #coordinata x del punto(landmark) più lontano dal naso
                ylont = y   #coordinata y del punto(landmark) più lontano dal naso



        #per ogni coppia di coordinate (x,y) all'interno dell'array di landmarks
        for(x,y) in shape:
            #settore = [0,0,0]
            if(y == ynose):
                m = 0
            else:
                m = (x - xnose)/(y-ynose)
            m = abs(m)

            #calcola la distanza euclidea di ogni punto(landmark) dal punto del naso
            d = distanza(xnose, ynose, x,y)

            
            #aggiunge il punto di coordinate x,y nell'array caratteristico
            aggiungi(xnose, ynose, raggio, x, y, d, m)

       
       
    dizionario[num_volto] = volto

    dizionario_str[num_volto] = img[0]

    num_volto = num_volto +1




  0%|          | 5/100186 [00:01<15:53:41,  1.75it/s]


## Salvataggio dati

In questa ultima sezione tutti i dati vengono salvati in un file csv. Si utilizzano differenti file a secondo del database

In [9]:
data=pd.DataFrame({'dizionario':dizionario,'dizionario_str':dizionario_str,'gender_str':gender_str, 'valid_face':valid_face})

data.to_csv('dfWheDataset.csv', index=False) #settaggio per WheDataset